In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 다차원에서 Transpose

In [0]:
import numpy as np

In [4]:
arr = np.arange(0, 4*2*4)
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [5]:
v = arr.reshape([4,2,4])
v

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]],

       [[24, 25, 26, 27],
        [28, 29, 30, 31]]])

In [7]:
v.ndim

3

In [8]:
v.sum()

496

In [10]:
print(v.sum(axis=0))    # axis=0 -> 행
print(v.shape)

[[48 52 56 60]
 [64 68 72 76]]
(4, 2, 4)


In [11]:
print(v.sum(axis=1))    # axis=1 -> 열
print(v.shape)

[[ 4  6  8 10]
 [20 22 24 26]
 [36 38 40 42]
 [52 54 56 58]]
(4, 2, 4)


In [12]:
print(v.sum(axis=2))    # axis=2 -> 깊이
print(v.shape)

[[  6  22]
 [ 38  54]
 [ 70  86]
 [102 118]]
(4, 2, 4)


## tensorflow의 transpose

In [13]:
myTensor = np.arange(2*3*4).reshape(2, 3, 4)
myTensor    # shape = (2,3,4)

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [15]:
print(np.transpose(myTensor))
print(np.transpose(myTensor).shape)    # shape = (4,3,2)

[[[ 0 12]
  [ 4 16]
  [ 8 20]]

 [[ 1 13]
  [ 5 17]
  [ 9 21]]

 [[ 2 14]
  [ 6 18]
  [10 22]]

 [[ 3 15]
  [ 7 19]
  [11 23]]]
(4, 3, 2)


In [16]:
np.transpose(myTensor, [1, 2, 0]).shape    # 대괄호안의 숫자는 원래 축의 axis number

(3, 4, 2)

# Tensorflow를 이용한 RNN 모델링으로 MNIST 데이터 분류하기

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [49]:
mnist = input_data.read_data_sets("./mnist2/data/", one_hot=True)

Extracting ./mnist2/data/train-images-idx3-ubyte.gz
Extracting ./mnist2/data/train-labels-idx1-ubyte.gz
Extracting ./mnist2/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist2/data/t10k-labels-idx1-ubyte.gz


In [0]:
learning_rate = 0.001
total_epoch = 30
batch_size = 128

In [0]:
n_input = 28    # 가로 픽셀 수
n_step = 28    # 세로 픽셀 수 (입력 단계)
n_hidden = 128
n_class = 10

In [0]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, n_step, n_input])
y = tf.placeholder(tf.float32, [None, n_class])
w = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [0]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

# Y: [batch_size, n_class]

# outputs: [batch_size, n_step, n_hidden]
# => [n_step, batch_size, n_hidden]
# => [batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])    # [n_step, batch_size, n_hidden]
outputs = outputs[-1]    # [batch_size, n_hidden]

# tf.matmul([batch_size, n_hidden], [n_hidden, n_class])
model = tf.matmul(outputs, w) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [59]:
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(total_epoch):
  total_cost = 0
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # [batch_size, n_step, n_input]
    batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
    _, cv = sess.run([optimizer, cost], feed_dict={x:batch_xs, y:batch_ys})
    total_cost += cv
  print("Epoch: %4d" % (epoch+1), ", avg cost: {:.3f}".format(total_cost/total_batch))

is_correct = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)

test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print("정확도: ", sess.run(accuracy, feed_dict={x:test_xs, y:test_ys}))

Epoch:    1 , avg cost: 0.512
Epoch:    2 , avg cost: 0.232
Epoch:    3 , avg cost: 0.178
Epoch:    4 , avg cost: 0.156
Epoch:    5 , avg cost: 0.144
Epoch:    6 , avg cost: 0.132
Epoch:    7 , avg cost: 0.116
Epoch:    8 , avg cost: 0.112
Epoch:    9 , avg cost: 0.111
Epoch:   10 , avg cost: 0.104
Epoch:   11 , avg cost: 0.099
Epoch:   12 , avg cost: 0.102
Epoch:   13 , avg cost: 0.086
Epoch:   14 , avg cost: 0.084
Epoch:   15 , avg cost: 0.089
Epoch:   16 , avg cost: 0.087
Epoch:   17 , avg cost: 0.078
Epoch:   18 , avg cost: 0.076
Epoch:   19 , avg cost: 0.075
Epoch:   20 , avg cost: 0.074
Epoch:   21 , avg cost: 0.079
Epoch:   22 , avg cost: 0.077
Epoch:   23 , avg cost: 0.073
Epoch:   24 , avg cost: 0.071
Epoch:   25 , avg cost: 0.062
Epoch:   26 , avg cost: 0.064
Epoch:   27 , avg cost: 0.067
Epoch:   28 , avg cost: 0.075
Epoch:   29 , avg cost: 0.063
Epoch:   30 , avg cost: 0.062
정확도:  0.9765


# 딕셔너리 만들기

In [61]:
# {"a": 0, ..., "z": 25} 딕셔너리 만들기

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

char_dict = {chr:i for i, chr in enumerate(char_arr)}
char_dict

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}